In [8]:
import pandas as pd

In [9]:
bores = pd.read_csv('NGIS_Bore.csv', usecols=[
    'HydroID',
    'HydroCode',
    'StateTerritory',
    'BoreDepth',
    'DrilledDate',
    'Latitude',
    'Longitude',
    'LandElev',
    'FTypeClass' # commerical, industrial, domestic etc..
])

# JOIN ON HYDRO ID ONLY???


In [10]:
bores = bores.rename(columns={
    'HydroID': 'hydro_id',
    'HydroCode': 'hydro_code',
    'StateTerritory': 'state',
    'BoreDepth': 'bore_depth',
    'DrilledDate': 'drilled_date',
    'Latitude': 'lat',
    'Longitude': 'long',
    'LandElev': 'land_elevation',
    'FTypeClass': 'type_of_use'
})

#bores.set_index(['bore_id', 'state', 'hydro_id'], inplace=True, verify_integrity=True)

In [11]:
bores.set_index('hydro_id', inplace=True, verify_integrity=True)

In [13]:
for state in ['NSW', 'QLD', 'NT', 'SA', 'TAS', 'VIC', 'WA']:
    print(f'Working on {state}')
    ## SALINITY
    salinity = pd.read_csv(f'{state}/salinity_{state}.csv', usecols=[
        'hydroid', 'bore_id', 'bore_date', 'uom', 'result'
    ])
    salinity = salinity.rename(columns={
        'hydroid': 'hydro_id',
        'bore_date': 'last_salinity_measurement'
    })
    # only take latest salinity measurements
    salinity = salinity.sort_values('last_salinity_measurement', ascending=False).drop_duplicates('hydro_id')
    salinity.set_index('hydro_id', inplace=True, verify_integrity=True)
    
    df = salinity.merge(bores, on='hydro_id', how='left')
    
    ## LEVELS
    levels = pd.read_csv(f'{state}/level_{state}.csv', usecols=[
        'hydroid', 'bore_id', 'bore_date', 'obs_point_datum', 'result'
    ]) 
    
    levels = levels.rename(columns={
        'hydroid': 'hydro_id',
        'bore_date': 'last_level_measurement'
    })
    
    levels = levels.sort_values('last_level_measurement', ascending=False).drop_duplicates('hydro_id')
    df2 = df.merge(levels, on='bore_id', how='left')
    # convert EC salinity readings (uS/cm, EC) to TDS salinity readings
    df2['result_x'] = df2.apply(lambda x: round(x['result_x'] * 0.55) if (x['uom'] == 'EC' or x['uom'] == 'uS/cm') else round(x['result_x']), axis=1)
    df2['uom'] = 'TDS'
    df2.to_csv(f'output_{state}.csv', index=False)
    #df2 = df2.fillna('')
    #df_to_geojson(state, df2)

    
    

Working on NSW
Working on QLD
Working on NT
Working on SA
Working on TAS
Working on VIC
Working on WA


In [16]:
opt1 = df2[df2.obs_point_datum == 'SWL']
opt2 = df2[df2.obs_point_datum == 'RSWL (mAHD)']
opt3 = df2[df2.obs_point_datum == 'DTW']

In [22]:
for value in opt3.result_y.values:
    print(value)

2.23
2.47
2.58
2.32
1.86
1.62
0.9
1.58
0.74
0.85
0.86
2.09
2.62
2.64
2.91
2.83
3.83
1.81
1.19
2.89
1.55
1.59
1.65
1.73
1.87
1.58
1.73
1.75
0.4
0.55
0.92
0.58
0.73
1.88
0.94
0.38
0.55
0.85
0.2
0.1
0.93
0.97
2.42
3.05
0.8
1.35
1.52
0.72
0.3
1.59
0.2
19.11
22.31
67.64
65.27
27.05
33.45
7.53
9.8
15.84
4.82


In [69]:
df2.result_x

0       14677
1          10
2         565
3         803
4        1068
        ...  
6202      251
6203      550
6204      656
6205      880
6206      128
Name: result_x, Length: 6207, dtype: int64

In [47]:
#df2[(df2.uom == 'EC') | (df2.uom == 'uS/cm')] 
df2['result_x'] = df2.apply(lambda x: x['result_x'] * 0.55 if (x['uom'] == 'EC' or x['uom'] == 'uS/cm') else x['result_x'], axis=1)

    


In [70]:
df2

,bore_id,last_salinity_measurement,uom,result_x,hydro_code,state,bore_depth,drilled_date,lat,long,land_elevation,type_of_use,hydro_id,last_level_measurement,obs_point_datum,result_y
0,61611782,2019-06-17 00:00:00,TDS,14677,61611782,5,,1984/05/18 00:00:00,-32.001899,115.805793,1.91,Monitoring,5.01016e+07,1998-09-29 11:31:00,SWL,2.16
1,61611773,2019-06-17 00:00:00,TDS,10,61611773,5,,1984/06/22 00:00:00,-31.990600,115.786300,2.67,Monitoring,5.00152e+07,1998-09-29 10:18:00,SWL,1.2
2,61611775,2019-06-17 00:00:00,TDS,565,61611775,5,,1984/06/22 00:00:00,-31.990600,115.786300,2.67,Monitoring,5.00394e+07,1998-09-29 10:30:00,RSWL (mAHD),0.62
3,61611776,2019-06-17 00:00:00,TDS,803,61611776,5,,1984/06/22 00:00:00,-31.990600,115.786300,2.67,Monitoring,5.00293e+07,1998-09-29 10:37:00,SWL,2.17
4,61611777,2019-06-17 00:00:00,TDS,1068,61611777,5,,1984/06/22 00:00:00,-31.990600,115.786300,2.67,Monitoring,5.00222e+07,1998-09-29 10:44:00,RSWL (mAHD),0.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6202,61618465,1900-01-01 00:00:00,TDS,251,61618465,5,29,1900/01/01 00:00:00,-32.100041,115.903070,34,Stock and Domestic,,,,
6203,70810687,1900-01-01 00:00:00,TDS,550,70810687,5,,1900/01/01 00:00:00,-22.628323,120.094851,421.91,Stock and Domestic,,,,
6204,61618497,1900-01-01 00:00:00,TDS,656,61618497,5,27.1,1900/01/01 00:00:00,-31.708174,115.814212,57.5,Monitoring,,,,
6205,61671041,1900-01-01 00:00:00,TDS,880,61671041,5,716.4,2004/01/28 00:00:00,-31.953535,115.782978,16.15,Unknown,,,,


In [49]:
df2['uom'] = 'TDS'

In [50]:
df2

,hydro_id_x,bore_id,last_salinity_measurement,uom,result_x,hydro_code,state,bore_depth,drilled_date,lat,long,land_elevation,type_of_use,hydro_id_y,last_level_measurement,obs_point_datum,result_y
0,10097525,GW080073.1.1,2019-06-26 00:00:00,TDS,304.70,GW080073.1.1,1,18.0,2000/06/09 00:00:00,-32.036200,150.855300,207.346,Monitoring,10097525.0,2019-09-01 00:00:00,SWL,10.25
1,10159509,GW075409.2.2,2019-06-26 00:00:00,TDS,121.00,GW075409.2.2,1,100.0,2011/05/31 00:00:00,-34.230397,150.543889,307.360,Monitoring,10159509.0,2019-08-27 00:00:00,SWL,16.73
2,10102662,GW080838.1.1,2019-06-26 00:00:00,TDS,4588.65,GW080838.1.1,1,NaN,NaN,-32.081299,151.984711,111.048,Unknown,10102662.0,2019-09-01 00:00:00,SWL,4.33
3,10104801,GW080843.1.1,2019-06-26 00:00:00,TDS,1851.30,GW080843.1.1,1,NaN,NaN,-32.042597,151.967783,100.713,Unknown,10104801.0,2019-09-01 00:00:00,SWL,6.38
4,10159605,GW075411.1.1,2019-06-26 00:00:00,TDS,129.25,GW075411.1.1,1,27.5,2011/06/22 00:00:00,-34.217500,150.549167,306.990,Monitoring,10159605.0,2019-09-01 00:00:00,SWL,13.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22640,10051156,GW045546.1.1,1916-01-01 00:00:00,TDS,1083.00,GW045546.1.1,1,123.4,1916/01/01 00:00:00,-32.071489,147.839433,NaN,Water Supply,NaN,NaN,NaN,NaN
22641,10002816,GW005290.1.1,1914-01-01 00:00:00,TDS,1112.00,GW005290.1.1,1,73.2,1914/01/01 00:00:00,-31.956850,147.786369,NaN,Stock and Domestic,NaN,NaN,NaN,NaN
22642,10031982,GW004208.1.1,1908-01-01 00:00:00,TDS,665.00,GW004208.1.1,1,250.6,1908/01/01 00:00:00,-30.336779,147.286181,126.200,Stock and Domestic,NaN,NaN,NaN,NaN
22643,10021860,GW048671.1.1,1908-01-01 00:00:00,TDS,1357.00,GW048671.1.1,1,78.0,1908/01/01 00:00:00,-32.008997,147.776469,NaN,Stock and Domestic,NaN,NaN,NaN,NaN


In [56]:
import geojson

In [57]:
def df_to_geojson(name, df):
    old_df = df
    features = []
    simple_features = []
    insert_features = lambda X: features.append(
            geojson.Feature(geometry=geojson.Point((X["long"],
                                                    X["lat"])),
                            properties=dict(id=X["bore_id"], 
                                            icon="drinking-water-15", 
                                            last_salinity_measurement=X["last_salinity_measurement"],
                                            salinity_uom=X["uom"],
                                            salinity=X["result_x"],
                                            bore_depth=X["bore_depth"],
                                            drilled_date=X["drilled_date"],
                                            land_elevation=X["land_elevation"],
                                            type_of_use=X["type_of_use"],
                                            last_level_measurement=X["last_level_measurement"],
                                            obs_point_datum=X["obs_point_datum"],
                                            level=X["result_y"]
                                           )))
    insert_features_simple = lambda X: simple_features.append(
        geojson.Feature(geometry=geojson.Point((X["long"],
                                                X["lat"])),
                        properties=dict(id=X["bore_id"], icon="drinking-water-15")
                       )
    )

    df.apply(insert_features, axis=1)
    old_df.apply(insert_features_simple, axis=1)
    

    with open(f'{name}.geojson', 'w', encoding='utf8') as fp:
        geojson.dump(geojson.FeatureCollection(features), fp, sort_keys=True, ensure_ascii=False)
        
    with open(f'{name}_simple.geojson', 'w', encoding='utf8') as fp:
        geojson.dump(geojson.FeatureCollection(simple_features), fp, sort_keys=True, ensure_ascii=False)

In [9]:
import pandas as pd
import geojson

In [21]:
df = pd.read_csv('Heat_map_data.csv')
df = df.fillna('')

In [22]:
df = df.sample(n=100000)

In [23]:
len(df.index)

100000

In [24]:
insert_features_simple = lambda X: features.append(
    geojson.Feature(geometry=geojson.Point((X["longitude"],
                                            X["latitude"])),
                    properties=dict(id=X["FID"], temp=X["temp"], sal=X["psal"])
                   )
)
features =[]

In [25]:
df.apply(insert_features_simple, axis=1)    
with open('heatmap.geojson', 'w', encoding='utf8') as fp:
        geojson.dump(geojson.FeatureCollection(features), fp, sort_keys=True, ensure_ascii=False)